In [1]:
!pip install transformers
!pip install 'accelerate>=0.26.0'
!pip install datasets


[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


In [1]:
import pandas as pd

In [2]:
import torch
from torch.utils.data import TensorDataset, DataLoader

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [3]:
import pandas as pd

train_df_raw = pd.read_csv('../../train.csv')
test_df_raw = pd.read_csv('../../test.csv')

# Prepare the ensemble

In [5]:
Models = ['BERT', 'CNN', 'Doc2Vec', 'LSTM', 'RNN']

dfs = []

for model in Models:
    dfs.append(pd.read_csv(f'../{model}/Results/train.csv'))

combined_df = pd.concat(
    [df['predicted_similarity'].reset_index(drop=True) for df in dfs],
    axis=1
)
combined_df.columns = Models

train_df = pd.concat(
    [train_df_raw, combined_df] ,
    axis=1
)

train_df

,id,s1,s2,score,BERT,CNN,Doc2Vec,LSTM,RNN
0,sts_train_4024,Sudan Blocks YouTube Over Anti-Islam Film,Pakistan's PM orders YouTube halt over anti-Is...,0.56,0.556423,0.573219,0.374772,0.532320,0.630532
1,sts_train_1409,A man riding a white horse.,A woman leading a white horse.,0.36,0.334312,0.494446,0.431345,0.354465,0.460784
2,sts_train_3397,Mr Morse is charged with assault and Mr Darvis...,His partner Bijan Darvish is charged with fili...,0.55,0.469501,0.585290,0.602250,0.556305,0.500478
3,sts_train_1532,A girl playing is a pile of colorful balls.,A little girl plays in a pit of colorful balls.,1.00,1.008758,0.955592,0.878527,0.980987,0.911502
4,sick_train_1874,A person in a black jacket is doing tricks on ...,A man in a black jacket is doing tricks on a m...,0.98,0.943288,0.966652,0.724846,0.973854,0.960750
...,...,...,...,...,...,...,...,...,...
12828,sick_train_6323,A player of soccer is scoring a goal,Soccer players are kicking a soccer ball into ...,0.86,0.823367,0.766068,0.600554,0.855742,0.956778
12829,sick_train_87,The man and woman are strolling,The man and woman are walking,0.90,0.919968,0.893222,0.670413,0.891996,0.763841
12830,sts_train_3897,S. Africa's former president Mandela discharge...,Mandela Discharged From Hospital,0.84,0.864671,0.897016,0.527709,0.810817,0.729604
12831,sick_train_718,A large brown cat is jumping over a red hurdle,A large brown dog is jumping over a red hurdle,0.68,0.742021,0.651751,0.863331,0.694979,0.733784


In [7]:
dfs = []

for model_names in Models:
    dfs.append(pd.read_csv(f'../{model_names}/Results/test.csv'))

combined_df = pd.concat(
    [df['predicted_similarity'].reset_index(drop=True) for df in dfs],
    axis=1
)
combined_df.columns = Models

test_df = pd.concat(
    [test_df_raw, combined_df],
    axis=1
)

test_df

,id,s1,s2,score,BERT,CNN,Doc2Vec,LSTM,RNN
0,sick_test_822,A woman is talking to a man,A man is talking to a woman,0.90,0.974569,0.950129,0.703845,0.852713,0.913445
1,sts_test_111,A man is putting garlic on some bread slices.,A man is sprinkling seasoning on several split...,0.40,0.691742,0.775935,0.488734,0.778635,0.639946
2,sts_test_502,A bike is next to a couple women.,A child next to a bike.,0.40,0.765109,0.789641,0.325368,0.764466,0.708850
3,sick_test_1253,A man is playing a bamboo flute,A flute is being played by the man,0.86,0.964099,0.996158,0.657300,0.735184,0.957379
4,sick_test_1967,A woman on a rock is lying on a blanket and re...,A woman is rocking over a blanket lying on som...,0.54,0.946081,0.903650,0.724827,0.729991,0.917402
...,...,...,...,...,...,...,...,...,...
3342,sick_test_1099,A woman is collecting tap water in a mug,There is no boy filling a pitcher with water,0.52,0.498875,0.613817,0.578613,0.559423,0.239837
3343,sts_test_559,There is a cook preparing food.,A cook is making food.,1.00,0.987018,0.885129,0.571081,0.897953,0.704321
3344,sick_test_1212,A young boy is playing a guitar,A boy is playing guitar,0.86,0.898162,0.973016,0.782884,0.878593,0.983629
3345,sick_test_1101,"A dog, which is small, is playing on the green...","A dog is near a ball colored in red, which is ...",0.48,0.602705,0.801525,0.504830,0.521849,0.635977


## Specific to BERT

In [8]:
train_df = train_df[['id', 's1', 's2', 'score', 'BERT']]

train_df = train_df.reset_index(drop=True)
train_df.to_csv('./train_df.csv')
train_df

,id,s1,s2,score,BERT
0,sts_train_4024,Sudan Blocks YouTube Over Anti-Islam Film,Pakistan's PM orders YouTube halt over anti-Is...,0.56,0.556423
1,sts_train_1409,A man riding a white horse.,A woman leading a white horse.,0.36,0.334312
2,sts_train_3397,Mr Morse is charged with assault and Mr Darvis...,His partner Bijan Darvish is charged with fili...,0.55,0.469501
3,sts_train_1532,A girl playing is a pile of colorful balls.,A little girl plays in a pit of colorful balls.,1.00,1.008758
4,sick_train_1874,A person in a black jacket is doing tricks on ...,A man in a black jacket is doing tricks on a m...,0.98,0.943288
...,...,...,...,...,...
12828,sick_train_6323,A player of soccer is scoring a goal,Soccer players are kicking a soccer ball into ...,0.86,0.823367
12829,sick_train_87,The man and woman are strolling,The man and woman are walking,0.90,0.919968
12830,sts_train_3897,S. Africa's former president Mandela discharge...,Mandela Discharged From Hospital,0.84,0.864671
12831,sick_train_718,A large brown cat is jumping over a red hurdle,A large brown dog is jumping over a red hurdle,0.68,0.742021


In [9]:
test_df = test_df[['id', 's1', 's2', 'score', 'BERT']]

test_df = test_df.reset_index(drop=True)
test_df.to_csv('./test_df.csv')
test_df

,id,s1,s2,score,BERT
0,sick_test_822,A woman is talking to a man,A man is talking to a woman,0.90,0.974569
1,sts_test_111,A man is putting garlic on some bread slices.,A man is sprinkling seasoning on several split...,0.40,0.691742
2,sts_test_502,A bike is next to a couple women.,A child next to a bike.,0.40,0.765109
3,sick_test_1253,A man is playing a bamboo flute,A flute is being played by the man,0.86,0.964099
4,sick_test_1967,A woman on a rock is lying on a blanket and re...,A woman is rocking over a blanket lying on som...,0.54,0.946081
...,...,...,...,...,...
3342,sick_test_1099,A woman is collecting tap water in a mug,There is no boy filling a pitcher with water,0.52,0.498875
3343,sts_test_559,There is a cook preparing food.,A cook is making food.,1.00,0.987018
3344,sick_test_1212,A young boy is playing a guitar,A boy is playing guitar,0.86,0.898162
3345,sick_test_1101,"A dog, which is small, is playing on the green...","A dog is near a ball colored in red, which is ...",0.48,0.602705


In [6]:
train_df = pd.read_csv('./train_df.csv')
test_df = pd.read_csv('./test_df.csv')

In [19]:
from datasets import Dataset
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained('microsoft/MiniLM-L12-H384-uncased')


tokenizer_config.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/385 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [75]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(
    'microsoft/MiniLM-L12-H384-uncased',
    num_labels=1  # for regression
)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at microsoft/MiniLM-L12-H384-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [22]:
# compute the sequence length using 95% samples logic
lengths = []
for _, row in train_df.iterrows():
    lengths.append(len(row['s1']))
    lengths.append(len(row['s2']))

lengths.sort()
MAX_LEN = lengths[int(0.95*len(lengths))]
print(MAX_LEN)

106


In [17]:
import torch
from torch import nn
from transformers import Trainer, TrainingArguments

# Custom loss function
class DistillationLoss(nn.Module):
    def __init__(self, alpha=0.5):
        super().__init__()
        self.alpha = alpha
        self.mse = nn.MSELoss()

    def forward(self, outputs, labels, soft_labels):
        preds = outputs.logits.view(-1)
        loss_hard = self.mse(preds, labels)
        loss_soft = self.mse(preds, soft_labels)
        return self.alpha * loss_hard + (1 - self.alpha) * loss_soft


In [20]:
def convert_sentences_to_features(sentences, tokenizer, max_len):
    input_ids = []
    attention_masks = []
    token_type_ids = []

    for i in range(0, len(sentences), 2):
        encoded_dict = tokenizer.encode_plus(sentences[i], sentences[i+1], add_special_tokens=True, max_length=max_len, truncation=True, padding='max_length', return_attention_mask=True, return_tensors='pt', truncation_strategy='longest_first')
        input_ids.append(encoded_dict['input_ids'])
        attention_masks.append(encoded_dict['attention_mask'])
        token_type_ids.append(encoded_dict['token_type_ids'])

    input_ids = torch.cat(input_ids, dim=0)
    attention_masks = torch.cat(attention_masks, dim=0)
    token_type_ids = torch.cat(token_type_ids, dim=0)

    return input_ids, attention_masks, token_type_ids

In [18]:


# train_dataset = .map(preprocess_function, batched=False)
# test_dataset = test_dataset.map(preprocess_function, batched=False)

x_train = []

for _, row in train_df.iterrows():
    x_train.append(row['s1'])
    x_train.append(row['s2'])

input_ids, attention_masks, token_type_ids = convert_sentences_to_features(x_train, tokenizer, max_len=MAX_LEN)
y_train = torch.tensor(train_df['score'], dtype=torch.float)

soft_labels = torch.tensor(train_df['BERT'], dtype=torch.float)

data_dict = {
    'input_ids': input_ids.to(device),
    'attention_mask': attention_masks.to(device),
    'token_type_ids': token_type_ids.to(device),
    'labels': y_train.to(device),
    'soft_labels': soft_labels.to(device),
}

train_dataset = Dataset.from_dict(data_dict)

x_test = []

for _, row in test_df.iterrows():
    x_test.append(row['s1'])
    x_test.append(row['s2'])

input_ids, attention_masks, token_type_ids = convert_sentences_to_features(x_test, tokenizer, max_len=MAX_LEN)
y_test = torch.tensor(test_df['score'], dtype=torch.float)

soft_labels = torch.tensor(test_df['BERT'], dtype=torch.float)

data_dict = {
    'input_ids': input_ids.to(device),
    'attention_mask': attention_masks.to(device),
    'token_type_ids': token_type_ids.to(device),
    'labels': y_test.to(device),
    'soft_labels': soft_labels.to(device),
}

test_dataset = Dataset.from_dict(data_dict)

NameError: name 'tokenizer' is not defined

In [66]:
model = model.to(device)

In [76]:
from transformers import TrainerCallback

# Custom Trainer with distillation loss
class DistillationTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False, num_items_in_batch=None):
        # labels = inputs.pop("labels")
        # print(inputs)
        labels = inputs["labels"]  # ✅ keep this; Trainer ensures it's here
        # soft_labels = inputs.get("soft_labels", None)  # ✅ safe access
        # print(soft_labels)
        soft_labels = inputs.pop("soft_labels")
        outputs = model(**inputs)
        loss_fct = DistillationLoss(alpha=0.5)
        loss = loss_fct(outputs, labels, soft_labels)
        return (loss, outputs) if return_outputs else loss

training_args = TrainingArguments(
    output_dir="./tinybert-distilled",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=7,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    remove_unused_columns=False,
    report_to="none",
)

print(train_dataset)
print(test_dataset)

from transformers import DataCollatorWithPadding

trainer = DistillationTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    # data_collator=data_collator,
)


Dataset({
    features: ['input_ids', 'attention_mask', 'token_type_ids', 'labels', 'soft_labels'],
    num_rows: 12833
})
Dataset({
    features: ['input_ids', 'attention_mask', 'token_type_ids', 'labels', 'soft_labels'],
    num_rows: 3347
})


In [77]:
trainer.train()

Epoch,Training Loss,Validation Loss
1,0.032400,0.018016
2,0.012600,0.016369
3,0.010400,0.016943
4,0.007600,0.017203
5,0.006400,0.014932
6,0.005500,0.015179
7,0.004800,0.014810


TrainOutput(global_step=5621, training_loss=0.010552009427447778, metrics={'train_runtime': 669.8751, 'train_samples_per_second': 134.101, 'train_steps_per_second': 8.391, 'total_flos': 1225067459413476.0, 'train_loss': 0.010552009427447778, 'epoch': 7.0})

In [78]:
torch.save(trainer.model, 'models/model.pt')

In [13]:
model = torch.load('models/model.pt', map_location=torch.device('cpu'))
model.to(device)

/tmp/ipykernel_24691/3635517369.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model = torch.load('models/model.pt', map_location=torch.device('cpu'))


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 384, padding_idx=0)
      (position_embeddings): Embedding(512, 384)
      (token_type_embeddings): Embedding(2, 384)
      (LayerNorm): LayerNorm((384,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=384, out_features=384, bias=True)
              (key): Linear(in_features=384, out_features=384, bias=True)
              (value): Linear(in_features=384, out_features=384, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=384, out_features=384, bias=True)
              (LayerNorm): LayerNorm((384,), eps=1e

In [14]:
best_model = model

In [24]:
from tqdm.notebook import tqdm

testdata = pd.read_csv('test_df.csv')

x_test = []
for _, row in testdata.iterrows():
    x_test.append(row['s1'])
    x_test.append(row['s2'])

input_ids, attention_masks, token_type_ids = convert_sentences_to_features(x_test, tokenizer, MAX_LEN)
y_test = torch.tensor(testdata['score'], dtype=torch.float)

testset = TensorDataset(input_ids, attention_masks, token_type_ids, y_test)
testloader = DataLoader(testset, batch_size=16, shuffle=False)

df_raw = pd.read_csv('../../test.csv')
id = df_raw['id'].values
s1_raw = df_raw['s1'].values
s2_raw = df_raw['s2'].values
y_raw = df_raw['score'].values

y_pred = []
y_true = []

best_model.eval()
with torch.no_grad():
    for _, batch in tqdm(enumerate(testloader), total=len(testloader)):
        input_ids, attention_masks, _, labels = tuple(t for t in batch)
        outputs = best_model(input_ids.to(device), token_type_ids=None, attention_mask=attention_masks.to(device), labels=labels.to(device))
        y_true.extend(labels.tolist())
        y_pred.extend([row[0] for row in outputs[1].tolist()])
    # print(len(train_embeddings1))
    # output = model(train_embeddings1, train_embeddings2)
    # y_pred = output.squeeze()
    # # y_pred = model(s1_train, s2_train)
    # # y_pred = y_pred.numpy()
    # # y_pred = np.squeeze(y_pred)
    # # y_pred = np.clip(y_pred, 0, 5)
    # print(id.shape, s1_raw.shape, y_raw.shape, len(y_pred))
    results = pd.DataFrame({'id': id, 'sentence1': s1_raw, 'sentence2': s2_raw, 'similarity': y_raw, 'predicted_similarity': y_pred})
    results.to_csv(f'Results/test.csv', index=False)
    print(f'Predictions saved in results.csv')

correlation = pd.Series(y_raw).corr(pd.Series(y_pred))
print('Correlation between expected and predicted similarity scores:', correlation)


  0%|          | 0/210 [00:00<?, ?it/s]

Predictions saved in results.csv
Correlation between expected and predicted similarity scores: 0.8943392931119134


In [82]:
from tqdm.notebook import tqdm

traindata = pd.read_csv('train_df.csv')

x_train = []
for _, row in traindata.iterrows():
    x_train.append(row['s1'])
    x_train.append(row['s2'])

input_ids, attention_masks, token_type_ids = convert_sentences_to_features(x_train, tokenizer, MAX_LEN)
y_train = torch.tensor(traindata['score'], dtype=torch.float)

trainset = TensorDataset(input_ids, attention_masks, token_type_ids, y_train)
trainloader = DataLoader(trainset, batch_size=16, shuffle=False)

df_raw = pd.read_csv('./train.csv')
id = df_raw['id'].values
s1_raw = df_raw['s1'].values
s2_raw = df_raw['s2'].values
y_raw = df_raw['score'].values

y_pred = []
y_true = []

best_model.eval()
with torch.no_grad():
    for _, batch in tqdm(enumerate(trainloader), total=len(trainloader)):
        input_ids, attention_masks, _, labels = tuple(t for t in batch)
        outputs = best_model(input_ids.to(device), token_type_ids=None, attention_mask=attention_masks.to(device), labels=labels.to(device))
        y_true.extend(labels.tolist())
        y_pred.extend([row[0] for row in outputs[1].tolist()])
    # print(len(train_embeddings1))
    # output = model(train_embeddings1, train_embeddings2)
    # y_pred = output.squeeze()
    # # y_pred = model(s1_train, s2_train)
    # # y_pred = y_pred.numpy()
    # # y_pred = np.squeeze(y_pred)
    # # y_pred = np.clip(y_pred, 0, 5)
    # print(id.shape, s1_raw.shape, y_raw.shape, len(y_pred))
    results = pd.DataFrame({'id': id, 'sentence1': s1_raw, 'sentence2': s2_raw, 'similarity': y_raw, 'predicted_similarity': y_pred})
    results.to_csv(f'Results/train.csv', index=False)
    print(f'Predictions saved in results.csv')

correlation = pd.Series(y_raw).corr(pd.Series(y_pred))
print('Correlation between expected and predicted similarity scores:', correlation)


  0%|          | 0/803 [00:00<?, ?it/s]

Predictions saved in results.csv
Correlation between expected and predicted similarity scores: 0.972750803642151
